In [1]:
# notebook to illustrate co-occurrence graphs

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [4]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/simple_test/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/hillsborough/txt/", text_filename_pattern="*.txt")

content_directory =  data_sets/hillsborough/txt/
text_filename_pattern =  *.txt
self.documents populated =  19217


In [5]:
# load previously calculated relevance scores
words_by_relevance = tmt.index_relevance.get_words_by_relevance(cr.content_directory)
# top most relevant
top_words_by_relevance = words_by_relevance[:100]

In [6]:
top_words_by_relevance[:10]

,relevance
police,83.451991
raised,79.235280
ground,66.290401
actions,65.596721
document,63.932343
action,61.090630
material,57.589417
number,51.046630
indexer,50.493850
instructions,48.442706


In [7]:
# a set is more efficient when checking for membership
top_words_by_relevance_set = set(top_words_by_relevance.index.tolist())
top_words_by_relevance_set

{'action',
 'actions',
 'address',
 'alcohol',
 'ambulance',
 'april',
 'authority',
 'being',
 'briefly',
 'category',
 'chief',
 'constable',
 'control',
 'could',
 'crowd',
 'department',
 'disaster',
 'disorder',
 'document',
 'endorse',
 'evidence',
 'examined',
 'final',
 'football',
 'forest',
 'further',
 'ground',
 'grounds',
 'house',
 'incident',
 'include',
 'indexed',
 'indexer',
 'indicated',
 'information',
 'injured',
 'inquiry',
 'instructions',
 'justice',
 'letter',
 'liverpool',
 'local',
 'london',
 'manager',
 'master',
 'match',
 'material',
 'meeting',
 'midlands',
 'number',
 'office',
 'officer',
 'officers',
 'other',
 'people',
 'pitch',
 'please',
 'police',
 'premises',
 'public',
 'questionnaire',
 'raised',
 'reader',
 'readers',
 'receiver',
 'receivers',
 'reference',
 'relevant',
 'report',
 'safety',
 'satisfied',
 'saturday',
 'secretary',
 'sheffield',
 'should',
 'signature',
 'source',
 'south',
 'stadium',
 'stand',
 'statement',
 'station',
 's

In [8]:
# first clear index
tmt.index_cooccurrence.delete_matrix(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # only keep words with min length 5
    fl = tmt.word_processing.keep_words_min_length(el,5)
    
    # remove all except those in the top most relevant list
    gl = [word for word in fl if word in top_words_by_relevance_set]
    
    # update index
    tmt.index_cooccurrence.create_cooccurrence_matrix_for_document(cr.content_directory, document_name, gl, window=4)
    pass


processing  AGO000000010001.txt
processing  AGO000000020001.txt
processing  AGO000000030001.txt
processing  AGO000000040001.txt
processing  AGO000000050001.txt
processing  AGO000000060001.txt
processing  AGO000000070001.txt
processing  AGO000000080001.txt
processing  AGO000000090001.txt
processing  AGO000000100001.txt
processing  AGO000000110001.txt
processing  AGO000000120001.txt
processing  AGO000000130001.txt
processing  AGO000000140001.txt
processing  AGO000000150001.txt
processing  AGO000000160001.txt
processing  AGO000000180001.txt
processing  AGO000000200001.txt
processing  AGO000000210001.txt
processing  AGO000000220001.txt
processing  AGO000000230001.txt
processing  AGO000000240001.txt
processing  AGO000000250001.txt
processing  AGO000000260001.txt
processing  AGO000000270001.txt
processing  AGO000000280001.txt
processing  AGO000000290001.txt
processing  AGO000000300001.txt
processing  AGO000000310001.txt
processing  AGO000000320001.txt
processing  AGO000000330001.txt
processi

processing  CMS000000800001.txt
processing  CMS000000840001.txt
processing  CMS000000850001.txt
processing  CMS000000860001.txt
processing  CMS000000870001.txt
processing  CMS000000880001.txt
processing  CMS000000910001.txt
processing  CMS000000920001.txt
processing  CMS000000930001.txt
processing  CMS000000940001.txt
processing  CMS000000950001.txt
processing  CMS000000960001.txt
processing  CMS000000980001.txt
processing  CMS000000990001.txt
processing  CMS000001000001.txt
processing  CMS000001010001.txt
processing  CMS000001040001.txt
processing  CMS000001050001.txt
processing  CMS000001060001.txt
processing  CMS000001070001.txt
processing  CMS000001080001.txt
processing  CMS000001090001.txt
processing  CMS000001100001.txt
processing  CMS000001110001.txt
processing  CMS000001140001.txt
processing  CMS000001150001.txt
processing  CMS000001160001.txt
processing  CMS000001170001.txt
processing  CMS000001180001.txt
processing  CMS000001190001.txt
processing  CMS000001250001.txt
processi

processing  CMS000004090001.txt
processing  CMS000004110001.txt
processing  CMS000004130001.txt
processing  CMS000004140001.txt
processing  CMS000004150001.txt
processing  CMS000004160001.txt
processing  CMS000004180001.txt
processing  CMS000004190001.txt
processing  CMS000004200001.txt
processing  CMS000004210001.txt
processing  CMS000004220001.txt
processing  CMS000004230001.txt
processing  CMS000004240001.txt
processing  CMS000004250001.txt
processing  CMS000004260001.txt
processing  CMS000004270001.txt
processing  CMS000004280001.txt
processing  CMS000004290001.txt
processing  CMS000004300001.txt
processing  CMS000004310001.txt
processing  CMS000004320001.txt
processing  CMS000004330001.txt
processing  CMS000004340001.txt
processing  CMS000004360001.txt
processing  CMS000004370001.txt
processing  CMS000004380001.txt
processing  CMS000004400001.txt
processing  CMS000004410001.txt
processing  CMS000004420001.txt
processing  CMS000004430001.txt
processing  CMS000004440001.txt
processi

processing  CMS000007100001.txt
processing  CMS000007120001.txt
processing  CMS000007130001.txt
processing  CMS000007140001.txt
processing  CMS000007150001.txt
processing  CMS000007160001.txt
processing  CMS000007170001.txt
processing  CMS000007180001.txt
processing  CMS000007190001.txt
processing  CMS000007200001.txt
processing  CMS000007210001.txt
processing  CMS000007220001.txt
processing  CMS000007240001.txt
processing  CMS000007250001.txt
processing  CMS000007260001.txt
processing  CMS000007270001.txt
processing  CMS000007290001.txt
processing  CMS000007310001.txt
processing  CMS000007320001.txt
processing  CMS000007330001.txt
processing  CMS000007340001.txt
processing  CMS000007350001.txt
processing  CMS000007360001.txt
processing  CMS000007380001.txt
processing  CMS000007400001.txt
processing  CMS000007410001.txt
processing  CMS000007430001.txt
processing  CMS000007440001.txt
processing  CMS000007550001.txt
processing  CMS000007560001.txt
processing  CMS000007570001.txt
processi

processing  CMS000010310001.txt
processing  CMS000010320001.txt
processing  CMS000010330001.txt
processing  CMS000010340001.txt
processing  CMS000010350001.txt
processing  CMS000010360001.txt
processing  CMS000010370001.txt
processing  CMS000010380001.txt
processing  CMS000010390001.txt
processing  CMS000010400001.txt
processing  CMS000010410001.txt
processing  CMS000010420001.txt
processing  CMS000010430001.txt
processing  CMS000010440001.txt
processing  CMS000010450001.txt
processing  CMS000010460001.txt
processing  CMS000010470001.txt
processing  CMS000010480001.txt
processing  CMS000010500001.txt
processing  CMS000010510001.txt
processing  CMS000010520001.txt
processing  CMS000010530001.txt
processing  CMS000010540001.txt
processing  CMS000010550001.txt
processing  CMS000010560001.txt
processing  CMS000010570001.txt
processing  CMS000010580001.txt
processing  CMS000010590001.txt
processing  CMS000010600001.txt
processing  CMS000010620001.txt
processing  CMS000010630001.txt
processi

...

processing  SYP000134820001.txt
processing  SYP000134840001.txt
processing  SYP000134850001.txt
processing  SYP000134860001.txt
processing  SYP000134870001.txt
processing  SYP000134880001.txt
processing  SYP000134910001.txt
processing  SYP000134950001.txt
processing  SYP000134970001.txt
processing  SYP000134980001.txt
processing  SYP000134990001.txt
processing  SYP000135000001.txt
processing  SYP000135010001.txt
processing  SYP000135020001.txt
processing  SYP000135030001.txt
processing  SYP000135040001.txt
processing  SYP000135050001.txt
processing  SYP000135060001.txt
processing  SYP000135080001.txt
processing  SYP000135090001.txt
processing  SYP000135100001.txt
processing  SYP000135120001.txt
processing  SYP000135130001.txt
processing  SYP000135140001.txt
processing  SYP000135150001.txt
processing  SYP000135170001.txt
processing  SYP000135180001.txt
processing  SYP000135210001.txt
processing  SYP000135240001.txt
processing  SYP000135480001.txt
processing  SYP000135520001.txt
processi

processing  SYP000142480001.txt
processing  SYP000142490001.txt
processing  SYP000142500001.txt
processing  SYP000142510001.txt
processing  SYP000142520001.txt
processing  SYP000142530001.txt
processing  SYP000142540001.txt
processing  SYP000142550001.txt
processing  SYP000142560001.txt
processing  SYP000142570001.txt
processing  SYP000142580001.txt
processing  SYP000142590001.txt
processing  SYP000142600001.txt
processing  SYP000142610001.txt
processing  SYP000142620001.txt
processing  SYP000142630001.txt
processing  SYP000142640001.txt
processing  SYP000142660001.txt
processing  SYP000142690001.txt
processing  SYP000142700001.txt
processing  SYP000142710001.txt
processing  SYP000142720001.txt
processing  SYP000142800001.txt
processing  SYP000142810001.txt
processing  SYP000142820001.txt
processing  SYP000142830001.txt
processing  SYP000142840001.txt
processing  SYP000142850001.txt
processing  SYP000142860001.txt
processing  SYP000142870001.txt
processing  SYP000142880001.txt
processi

processing  SYP000145540001.txt
processing  SYP000145550001.txt
processing  SYP000145560001.txt
processing  SYP000145570001.txt
processing  SYP000145580001.txt
processing  SYP000145600001.txt
processing  SYP000145610001.txt
processing  SYP000145620001.txt
processing  SYP000145630001.txt
processing  SYP000145640001.txt
processing  SYP000145650001.txt
processing  SYP000145660001.txt
processing  SYP000145670001.txt
processing  SYP000145680001.txt
processing  SYP000145690001.txt
processing  SYP000145710001.txt
processing  SYP000145720001.txt
processing  SYP000145730001.txt
processing  SYP000145740001.txt
processing  SYP000145750001.txt
processing  SYP000145760001.txt
processing  SYP000145770001.txt
processing  SYP000145780001.txt
processing  SYP000145790001.txt
processing  SYP000145800001.txt
processing  SYP000145810001.txt
processing  SYP000145820001.txt
processing  SYP000145830001.txt
processing  SYP000145840001.txt
processing  SYP000145850001.txt
processing  SYP000145860001.txt
processi

processing  TPF000000330001.txt
processing  TRH000000010001.txt
processing  TRH000000020001.txt
processing  TRH000000030001.txt
processing  TRH000000040001.txt
processing  TRH000000050001.txt
processing  TRH000000060001.txt
processing  TRH000000070001.txt
processing  TRH000000080001.txt
processing  TRH000000090001.txt
processing  TRH000000100001.txt
processing  TRH000000110001.txt
processing  TRH000000120001.txt
processing  TRH000000130001.txt
processing  TRH000000140001.txt
processing  TRH000000160001.txt
processing  TRH000000170001.txt
processing  TSO000000010001.txt
processing  TSO000000020001.txt
processing  TSO000000030001.txt
processing  TSO000000040001.txt
processing  TSO000000050001.txt
processing  TSO000000080001.txt
processing  TSO000000110001.txt
processing  TSO000000150001.txt
processing  TSO000000170001.txt
processing  TSO000000180001.txt
processing  TSO000000190001.txt
processing  TSO000000200001.txt
processing  TSO000000210001.txt
processing  TSO000000250001.txt
processi

processing  YAS000001180001.txt
processing  YAS000001190001.txt
processing  YAS000001230001.txt
processing  YAS000001240001.txt
processing  YAS000001250001.txt
processing  YAS000001260001.txt
processing  YAS000001270001.txt
processing  YAS000001280001.txt
processing  YAS000001290001.txt
processing  YAS000001300001.txt
processing  YAS000001310001.txt
processing  YAS000001320001.txt
processing  YAS000001330001.txt
processing  YAS000001340001.txt
processing  YAS000001350001.txt
processing  YAS000001360001.txt
processing  YAS000001370001.txt
processing  YAS000001380001.txt
processing  YAS000001390001.txt
processing  YAS000001400001.txt
processing  YAS000001410001.txt
processing  YAS000001420001.txt
processing  YAS000001430001.txt
processing  YAS000001440001.txt
processing  YAS000001450001.txt
processing  YAS000001460001.txt
processing  YAS000001470001.txt
processing  YAS000001480001.txt
processing  YAS000001490001.txt
processing  YAS000001500001.txt
processing  YAS000001510001.txt
processi

In [9]:
# merge document indices into a corpus index
tmt.index_cooccurrence.merge_cooccurrence_matrices_for_corpus(cr.content_directory)

data_sets/hillsborough/txt/AGO000000010001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000020001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000030001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000040001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000050001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000060001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000070001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000080001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000090001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000100001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000110001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000120001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000130001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000140001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000000150001.txt_matrix.cooccurr

data_sets/hillsborough/txt/AGO000002030001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002040001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002050001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002170001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002180001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002190001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002210001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002230001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002250001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002260001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002270001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002280001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002290001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002310001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/AGO000002320001.txt_matrix.cooccurr

data_sets/hillsborough/txt/CMS000000800001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000840001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000850001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000860001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000870001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000880001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000910001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000920001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000930001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000940001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000950001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000960001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000980001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000000990001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000001000001.txt_matrix.cooccurr

data_sets/hillsborough/txt/CMS000002570001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002580001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002590001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002600001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002610001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002620001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002630001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002640001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002650001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002660001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002670001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002680001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002690001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002700001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000002710001.txt_matrix.cooccurr

data_sets/hillsborough/txt/CMS000004240001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004250001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004260001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004270001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004280001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004290001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004300001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004310001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004320001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004330001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004340001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004360001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004370001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004380001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/CMS000004400001.txt_matrix.cooccurr

...

data_sets/hillsborough/txt/SYP000146520001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146530001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146540001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146550001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146560001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146570001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146580001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146590001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146600001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146610001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146620001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146630001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146640001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146650001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000146660001.txt_matrix.cooccurr

data_sets/hillsborough/txt/SYP000160160001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160170001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160180001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160190001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160200001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160210001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160220001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160230001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160240001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160250001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160260001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160270001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/SYP000160280001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/TPF000000010001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/TPF000000020001.txt_matrix.cooccurr

data_sets/hillsborough/txt/WYC000000250001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000260001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000270001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000280001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000290001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000300001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000310001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000320001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000340001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000350001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000360001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000370001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000380001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000400001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/WYC000000410001.txt_matrix.cooccurr

data_sets/hillsborough/txt/YAS000000880001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000890001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000900001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000910001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000920001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000930001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000940001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000950001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000960001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000970001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000980001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000000990001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000001000001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000001010001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000001020001.txt_matrix.cooccurr

data_sets/hillsborough/txt/YAS000002220001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002230001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002240001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002250001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002270001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002280001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002360001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002370001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002380001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002390001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002400001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002440001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002450001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002460001.txt_matrix.cooccurrence
data_sets/hillsborough/txt/YAS000002470001.txt_matrix.cooccurr

In [10]:
tmt.index_cooccurrence.print_matrix(cr.content_directory)

cooccurrence_matrix_file  data_sets/hillsborough/txt/matrix_cooccurrence.hdf5
                 action      actions     ...        yorkshire       yours
action     44489.642143  7147.287960     ...       774.852536  237.424633
actions    13391.702767   594.894381     ...       186.390964   48.833018
address     1542.122624   648.460854     ...      1038.884278  239.341241
alcohol       90.663330    26.340878     ...        82.733487   28.026666
ambulance    245.068993    67.173475     ...      3337.471403  102.399700
april        408.095887    98.652799     ...      1258.583817  529.687244
authority   1001.792098   237.909610     ...      1875.594445  536.841214
being        997.997031   474.057060     ...       974.231819  480.924213
briefly      159.764868    17.487494     ...        25.695400    6.710492
category      29.391341    24.461001     ...        28.064455    3.434677

[10 rows x 100 columns]


In [11]:
# query the co-occurrence value for a given pair of words
tmt.index_cooccurrence.query_cooccurance_matrix(cr.content_directory, "chief", "police")

13564.378193655819

In [12]:
# get the most likely next word by co-occurrence value for given word
tmt.index_cooccurrence.most_likely_next(cr.content_directory, "chief")

'constable'

In [13]:
#generate sentences based on co-occurence
sentence_length = 7

#initial words 
#for w in ["the", "once", "then", "olive", "tomato", "cut", "cook", "little", "boiled"]:
for w in ["evidence"]:
    sentence = w
    for i in range(sentence_length):
        w = tmt.index_cooccurrence.most_likely_next(cr.content_directory, w)
        sentence += (" " + w)
        pass
    print(sentence)
    pass


evidence evidence evidence evidence evidence evidence evidence evidence


In [5]:
words_by_co_occurrence = tmt.index_cooccurrence.get_word_pairs_by_cooccurrence(cr.content_directory)

In [6]:
# 20 most common
words_by_co_occurrence[:20]

,word1,word2,weight
0,police,police,1.000000
1,there,there,0.673316
2,ground,ground,0.650661
3,south,yorkshire,0.595835
4,would,would,0.568905
5,should,should,0.442588
6,material,police,0.422234
7,police,officers,0.418765
8,action,action,0.403173
9,which,which,0.402548


In [7]:
# 20 least common
words_by_co_occurrence[-20:]

,word1,word2,weight
9849,receivers,alcohol,0.000003
9850,include,indexed,0.000003
9851,indexed,signature,0.000003
9852,signature,urgent,0.000003
9853,receiver,yours,0.000003
9854,public,receivers,0.000003
9855,receivers,signature,0.000003
9856,receivers,liverpool,0.000003
9857,receivers,evidence,0.000003
9858,ambulance,indexed,0.000003


In [9]:
# visualise co-occurrence of words
tmt.visualisation.plot_force_directed_graph(words_by_co_occurrence[:500])

force-directed graph

In [10]:
# visualise co-occurrence > 1
w = words_by_co_occurrence[words_by_co_occurrence['weight']>0.2]
tmt.visualisation.plot_force_directed_graph(w)

force-directed graph